In [ ]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras

import tensorflow as tf

In [ ]:
SIZE = 7
CHANNELS = 20

In [ ]:
npz = np.load('gomoku/models/7_20channel.npz')
train_vectors = npz['train_vectors']
train_p = npz['train_p']
train_q = npz['train_q']

In [ ]:
inp = Input(shape=(SIZE, SIZE, CHANNELS))

# key difference between this and conv network is padding
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(inp)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(32, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_normal', use_bias=False)(bn2)
#bn3 = BatchNormalization()(conv_2)
#conv_3 = Convolution2D(32, (3, 3), padding='valid', activation='relu',
#                       kernel_initializer='random_normal', use_bias=False)(bn3)
#bn4 = BatchNormalization()(conv_3)
#conv_4 = Convolution2D(16, (3, 3), padding='valid', activation='relu',
#                       kernel_initializer='random_normal', use_bias=False)(bn4)
#bn5 = BatchNormalization()(conv_4)

flat = Flatten()(bn2)

hidden = Dense(10, activation='relu', kernel_initializer='random_normal', use_bias=False)(flat)
bn_final = BatchNormalization()(hidden)

out = Dense(1, use_bias=False)(bn_final)

q_model = Model(inputs=[inp], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])


In [ ]:
boards = np.array([board for board in train_vectors[:, 0]]).reshape(train_vectors.shape[0], SIZE, SIZE, CHANNELS)

In [ ]:
q_model.fit([boards, 
                train_vectors[:, 1]], 
                train_q, 
                shuffle=False,
              epochs=10,
             validation_split=0.1)

In [ ]:
q_model.save('models/batch_9_9_q_v3_5.net')

In [ ]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (5, 5), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)
#conv_4 = Convolution2D(64, (3, 3), padding='same', activation='relu',
#                       kernel_initializer='random_uniform')(bn4)
#bn5 = BatchNormalization()(conv_4)
#conv_5 = Convolution2D(64, (3, 3), padding='same', activation='relu',
#                       kernel_initializer='random_uniform')(bn5)
#bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn4)

p_model = Model(inputs=[inp, turn_input], outputs=out)
p_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    p_model.fit([boards, 
               train_vectors[:, 1]], 
              train_p, 
              epochs=5,
             validation_split=0.1)

In [ ]:
p_model.save('models/9_9_p_v2_2.net')